In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
import plotly.graph_objects as go
from database_tools.preprocessing.functions import bandpass, find_peaks, beat_similarity
from database_tools.preprocessing.utils import resample_signal

pd.options.plotting.backend = 'plotly'

x = 37

with open(f'data/TX{x}DATAPPG0.txt', 'r') as f:
    ppg_i_data = [l.strip('\n') for l in f.readlines()]

with open(f'data/TX{x}DATAPPG1.txt', 'r') as f:
    ppg_r_data = [l.strip('\n') for l in f.readlines()]

ppg_i = np.array(ppg_i_data[4::]).astype(int)
ppg_i_filt = bandpass(ppg_i, low=0.5, high=8.0, fs=200)
ppg_i_flip = np.max(ppg_i_filt) - ppg_i_filt

ppg_r = np.array(ppg_r_data[4::]).astype(int)
ppg_r_filt = bandpass(ppg_r, low=0.5, high=8.0, fs=200)
ppg_r_flip = np.max(ppg_r_filt) - ppg_r_filt

combined = (ppg_r_flip + ppg_i_flip) / 2

fig = go.Figure()

fig.add_scatter(y=combined, name='combined')

In [ ]:
x = resample_signal(combined, fs_old=200, fs_new=125)
xx = x[0:256]
xxx = np.pad(xx, 40, constant_values=np.mean(xx))
peaks, troughs = find_peaks(xxx).values()

fig = go.Figure()

fig.add_scatter(y=xxx)
fig.add_scatter(x=peaks, y=xxx[peaks], mode='markers')
fig.add_scatter(x=troughs, y=xxx[troughs], mode='markers')

In [ ]:
xx = x[256:512]
beats = beat_similarity(xx, fs=125, return_beats=False)
beats

In [ ]:
fig = go.Figure()

for b in beats:
    fig.add_scatter(y=b)
fig.show()

In [ ]:
peaks

In [ ]:
import os
import time
import numpy as np
import plotly.graph_objects as go
from database_tools.preprocessing.spo2 import calc_spo2, find_peaks
from database_tools.preprocessing.functions import bandpass

i = 34
ppg0 = f'data/TX{i}DATAPPG0.txt'
ppg1 = f'data/TX{i}DATAPPG1.txt'
while True:
    if os.path.exists(ppg1):
        with open(f'data/TX{i}DATAPPG0.txt', 'r') as f:
            ppg_i_data = [l.strip('\n') for l in f.readlines()]
        with open(f'data/TX{i}DATAPPG1.txt', 'r') as f:
            ppg_r_data = [l.strip('\n') for l in f.readlines()]
        i += 1
        ppg0 = f'data/TX{i}DATAPPG0.txt'
        ppg1 = f'data/TX{i}DATAPPG1.txt'

        ppg_i = np.array(ppg_i_data[4::]).astype(int)
        ppg_r = np.array(ppg_r_data[4::]).astype(int)

        fig = go.Figure()
        fig.add_scatter(y=ppg_i, name='IR Filtered')
        fig.add_scatter(y=ppg_r, name='Red Filtered')
        fig.show()

        results = {'spo2': [], 'r': []}
        for j in range(4):
            idx = (j*1025, (j+1)*1025)
            spo2, r = calc_spo2(ppg_r[idx[0]:idx[1]], ppg_i[idx[0]:idx[1]], return_peaks=False)
            results['spo2'].append(spo2)
            results['r'].append(r)

        ppg_i_flip = np.max(ppg_i) - ppg_i
        ppg_i_filt = bandpass(ppg_i_flip, low=0.5, high=8.0, fs=200)

        ppg_r_flip = np.max(ppg_r) - ppg_r
        ppg_r_filt = bandpass(ppg_r_flip, low=0.5, high=8.0, fs=200)
        combined = (ppg_i_filt + ppg_r_filt) / 2

        peaks = find_peaks(combined)
        heart_rate = int(round(len(peaks['peaks']) * 2.926829268292683, 0))
        print(results, heart_rate)
    else:
        time.sleep(5)

In [ ]:
fig = go.Figure()

ir_peaks = peaks['ir_peaks']
ir_troughs = peaks['ir_troughs']

fig.add_scatter(y=ppg_i[i[0]:i[1]])
fig.add_scatter(x=ir_peaks, y=ppg_i[i[0]:i[1]][ir_peaks], mode='markers')
fig.add_scatter(x=ir_troughs, y=ppg_i[i[0]:i[1]][ir_troughs], mode='markers')

In [ ]:
import pickle as pkl
from scipy import signal
from database_tools.preprocessing.utils import window

x = 41

with open(f'data/TX{x}DATAPPG0.txt', 'r') as f:
    ppg_i_data = [l.strip('\n') for l in f.readlines()]

with open(f'data/TX{x}DATAPPG1.txt', 'r') as f:
    ppg_r_data = [l.strip('\n') for l in f.readlines()]

ppg_i = np.array(ppg_i_data[4::]).astype(int)
ppg_i_flip = np.max(ppg_i) - ppg_i
ppg_i_filt = bandpass(ppg_i_flip, low=0.5, high=8.0, fs=200)

ppg_r = np.array(ppg_r_data[4::]).astype(int)
ppg_r_flip = np.max(ppg_r) - ppg_r
ppg_r_filt = bandpass(ppg_r_flip, low=0.5, high=8.0, fs=200)

combined = (ppg_i_filt + ppg_r_filt) / 2

fig = go.Figure()

fig.add_scatter(y=ppg_i_filt, name='IR Filtered')
fig.add_scatter(y=ppg_r_filt, name='Red Filtered')
fig.add_scatter(y=combined, name='combined')
fig.show()

print(np.min(combined), np.max(combined))
combined = np.divide(combined - np.min(combined), np.max(combined) - np.min(combined))
combined = signal.resample(combined, 2562)

idx = window(combined, win_len=256, overlap=0)

In [ ]:
idx_0 = idx[1]

ppg = combined[idx_0[0]:idx_0[1]].reshape(1, -1)
vpg = np.gradient(ppg, axis=1)  # 1st derivative of ppg
apg = np.gradient(vpg, axis=1) # 2nd derivative of vpg

with open('test-data.pkl', 'wb') as f:
    pkl.dump((ppg, vpg, apg), f)

In [ ]:
import pandas as pd

pd.Series(ppg.reshape(-1)).plot()

In [ ]:
round(0.98765 * 100, 1)